# Ray RLlib - Extra Application Example - Taxi-v3

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `Taxi-v3` environment ([gym.openai.com/envs/Taxi-v3/](https://gym.openai.com/envs/Taxi-v3/)). The goal is to pick up passengers as fast as possible, negotiating the available paths. This is one of OpenAI Gym's ["toy text"](https://gym.openai.com/envs/#toy_text) problems.

For more background about this problem, see:

* ["Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition"](https://arxiv.org/abs/cs/9905014), [Thomas G. Dietteric](https://twitter.com/tdietterich)
* ["Reinforcement Learning: let’s teach a taxi-cab how to drive"](https://towardsdatascience.com/reinforcement-learning-lets-teach-a-taxi-cab-how-to-drive-4fd1a0d00529), [Valentina Alto](https://twitter.com/AltoValentina)

In [7]:
import pandas as pd
import json, os, shutil, sys
import ray
import ray.rllib.agents.ppo as ppo

Let's start up Ray as in the previous lesson:

In [1]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [3]:
ray.init(ignore_reinit_error=True)

2020-06-13 13:58:24,019	INFO resource_spec.py:212 -- Starting Ray with 3.66 GiB memory available for workers and up to 1.85 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-13 13:58:24,350	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:28091',
 'object_store_address': '/tmp/ray/session_2020-06-13_13-58-24_007306_10900/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-13_13-58-24_007306_10900/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-06-13_13-58-24_007306_10900'}

In [4]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8266


Set up the checkpoint location:

In [9]:
checkpoint_root = 'tmp/ppo/taxi'
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Next we'll train an RLlib policy with the `Taxi-v3` environment.

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/taxi` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [5]:
SELECT_ENV = "Taxi-v3"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

2020-06-13 13:59:26,227	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-13 13:59:26,256	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-06-13 13:59:29,075	INFO trainable.py:217 -- Getting current IP.
2020-06-13 13:59:29,076	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [10]:
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')
reward_history = []

  1: Min/Mean/Max reward: -920.0000/-754.4667/-331.0000, len mean: 197.2167. Checkpoint saved to tmp/ppo/taxi/checkpoint_3/checkpoint-3
  2: Min/Mean/Max reward: -920.0000/-742.1875/-331.0000, len mean: 197.9125. Checkpoint saved to tmp/ppo/taxi/checkpoint_4/checkpoint-4
  3: Min/Mean/Max reward: -920.0000/-719.7300/-307.0000, len mean: 196.3500. Checkpoint saved to tmp/ppo/taxi/checkpoint_5/checkpoint-5
  4: Min/Mean/Max reward: -920.0000/-677.6200/-246.0000, len mean: 194.8900. Checkpoint saved to tmp/ppo/taxi/checkpoint_6/checkpoint-6
  5: Min/Mean/Max reward: -902.0000/-632.1200/-214.0000, len mean: 193.5800. Checkpoint saved to tmp/ppo/taxi/checkpoint_7/checkpoint-7
  6: Min/Mean/Max reward: -911.0000/-613.6000/-214.0000, len mean: 194.3800. Checkpoint saved to tmp/ppo/taxi/checkpoint_8/checkpoint-8
  7: Min/Mean/Max reward: -911.0000/-578.9500/-197.0000, len mean: 192.7300. Checkpoint saved to tmp/ppo/taxi/checkpoint_9/checkpoint-9
  8: Min/Mean/Max reward: -911.0000/-563.5800/-1

Do the episode rewards increase after multiple iterations?

Also, print out the policy and model to see the results of training in detail…

In [11]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(500, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(256, 256) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(256, 6) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(6,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(256, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
______

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

This visualizes the "taxi" agent operating within the simulation: picking up fares, driving, turning, doing dropoffs, etc.

In [15]:
!RAY_ADDRESS=auto rllib rollout \
    tmp/ppo/taxi/checkpoint_10/checkpoint-10 \
    --config "{\"env\": \"Taxi-v3\"}" --run PPO \
    --steps 2000

2020-06-13 14:05:34,892	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2020-06-13 14:05:34,974	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-13 14:05:35,045	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-06-13 14:05:37,350	INFO trainable.py:217 -- Getting current IP.
2020-06-13 14:05:37,351	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-06-13 14:05:37,418	INFO trainable.py:217 -- Getting current IP.
2020-06-13 14:05:37,418	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: tmp/ppo/taxi/c

## Exercise ("Homework")

In addition to _Taxi_, there are other so-called ["toy text"](https://gym.openai.com/envs/#toy_text) problems you can try.